<a href="https://colab.research.google.com/github/MichelaMarini/MATH-6373-PyTorch-tutorial/blob/main/WEEK_3_FNN_CLASSIFICATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import time

# Set a fixed random seed for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True

# Check if CUDA is available, otherwise, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create a synthetic dataset using make_circles
X, y = make_circles(n_samples=500, shuffle=True, noise=0.1, factor=0.7, random_state=42)


###PLOT THE DATASET (X)####
plt.figure(figsize=(6, 6))
plt.scatter(####, ####, c=###, cmap=#####, edgecolors=####, alpha=####)
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.title("Synthetic Dataset (make_circles)")
plt.show()


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

# Create datasets and data loaders
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 32)
        self.fc2 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.3)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# Function to calculate accuracy
def calculate_accuracy(net, valloader):
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in valloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs).squeeze()
            predictions = (outputs > 0.5).long()
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    return correct / total

# Training function
def train(net, trainloader, valloader, criterion, optimizer, num_epochs):
    train_losses, val_losses = [], []
    start_time = time.time()

    for epoch in range(num_epochs):
        running_train_loss = 0.0
        running_val_loss = 0.0
        net.train()

        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs.squeeze(), labels.float())
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item()

        net.eval()
        with torch.no_grad():
            for inputs, labels in valloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = net(inputs)
                val_loss = criterion(outputs.squeeze(), labels.float())
                running_val_loss += val_loss.item()

        epoch_train_loss = running_train_loss / len(trainloader)
        epoch_val_loss = running_val_loss / len(valloader)
        train_losses.append(epoch_train_loss)
        val_losses.append(epoch_val_loss)

        if (epoch + 1) % 10 == 0:
            print(f'[Epoch {epoch + 1}] Training loss: {epoch_train_loss:.3f} | Validation loss: {epoch_val_loss:.3f}')

    total_time = time.time() - start_time
    print(f"Total training time: {total_time:.2f} seconds")

    return train_losses, val_losses

batch_size = 8
num_epochs_list = [20, 60, 80]
fig, axs = plt.subplots(len(num_epochs_list), figsize=(8, 6))

for i, num_epochs in enumerate(num_epochs_list):
    print(f"Training with {num_epochs} epochs")
    net = Net().to(device)
    criterion = nn.BCELoss()
    optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
    trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    train_losses, val_losses = train(net, trainloader, valloader, criterion, optimizer, num_epochs)

    accuracy = calculate_accuracy(net, valloader)
    print(f"Validation Accuracy after {num_epochs} epochs: {accuracy:.2%}")

    axs[i].plot(train_losses, label="Train")
    axs[i].plot(val_losses, label="Validation")
    axs[i].set_xlabel('Epochs')
    axs[i].set_ylabel('Loss')
    axs[i].set_title(f'Training and Validation Loss vs Epochs (Epochs: {num_epochs})')
    axs[i].set_ylim(0, 1)
    axs[i].legend()

    model_filename = f"model_{num_epochs}_epochs.pth"
    torch.save(net.state_dict(), model_filename)
    print(f"Model saved as {model_filename}")

plt.tight_layout()
plt.show()


# Problem: Evaluating a Pre-Trained Model on a Test Dataset
In this exercise, you will evaluate a pre-trained neural network on a test dataset using PyTorch. Your task is to complete the given code to:


1.   **Generate a test dataset** using make_circles and convert it to PyTorch tensors.
2.   **Upload pre-trained models** (`model_20_epochs.pth`, `model_60_epochs.pth`, `model_80_epochs.pth`) and load them for evaluation.
3.   **Create a DataLoader for the test set** to efficiently process batches.
4.   **Evaluate the model** on the test dataset and compute its accuracy.


## **Instructions**


*   Carefully look for `# COMPLETE THIS PART #` and fill in the required code.
*   Ensure that the test dataset is correctly formatted before passing it to the model.
*   The model should output predictions in the range [0,1], so apply a threshold of 0.5 to convert them into binary class labels (0 or 1).
*    Verify that the accuracy calculation correctly compares predictions against ground truth labels.
*    Run the code and analyze the accuracy values for different trained models.

In [ ]:
# Generate test data
X_test, y_test = make_circles(n_samples=200, shuffle=True, noise=0.1, random_state=42, factor=0.7)

# Convert X_test and y_test to torch tensors
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=# COMPLETE THIS PART #)

# Load the saved model
def load_model(model_filename, net):
    net.load_state_dict(torch.load(model_filename, weights_only=True))
    # Set the model to evaluation mode
    net.# COMPLETE THIS PART #
    return # COMPLETE THIS PART #

# Evaluate the model on the test dataset
def evaluate_model(net, testloader, threshold=0.5):
    correct = 0
    total = 0
    with torch.no_grad():  # No need to compute gradients during evaluation
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = net(inputs).squeeze()  # Remove extra dimensions if needed

            # Convert to binary predictions
            predicted = (outputs > # COMPLETE THIS PART # ).long()

            # Update counters
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = # COMPLETE THIS PART #
    return accuracy

# Initialize the model
net = Net().to(device)

# Load and test the model for each different number of epochs
num_epochs_list = [20, 60, 80]

for num_epochs in num_epochs_list:
    model_filename = f"model_{num_epochs}_epochs.pth"
    print(f"Loading model from {model_filename}...")

    # Load the saved model
    net = load_model(# COMPLETE THIS PART # , # COMPLETE THIS PART # )

    # Create DataLoader for the test set
    test_dataset = TensorDataset(X_test, y_test)
    testloader = DataLoader(# COMPLETE THIS PART # , batch_size=200, shuffle=False)

    # Calculate accuracy
    # COMPLETE THIS PART #
    print(f"Accuracy of the model with {num_epochs} epochs: {accuracy:.2f}%")